# PyEumap - Land-Cover Mapping

In this tutorial, we will use the overlayed points (see [Overlay tutorial](02_overlay.ipynb)) to train a ML-model and predict the land-cover (LC) in the last two decades, using the **LandMapper** class.

The training step will use *elevation*, *slope*, *landsat* (7 spectral bands, 4 seasons and 3 percentiles per year) and *night light* (VIIRS Night Band) data to predict the follow LC classes:
* 211: Non-irrigated arable land
* 311: Broad-leaved forest
* 312: Coniferous forest
* 324: Transitional woodland-shrub
* 411: Inland wetlands
* 512: Water bodies

First, let's import the necessary modules

In [25]:
import sys
sys.path.append('../../')

import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
from pyeumap.mapper import LandMapper
from sklearn.ensemble import RandomForestClassifier

## Dataset

Our dataset refers to 1 tile, located in Sweden, extracted from a tiling system created for European Union (7,042 tiles) by GeoHarmonizer Project.

In [26]:
data_dir = '/home/leandro/Tmp/pilot_tiles/sweden_22497'

Let's load the overlayed points

In [27]:
fn_points = os.path.join(data_dir,'training_samples_overlayed.gpkg')
points = gpd.read_file(fn_points)
points

,lucas,survey_date,confidence,lc_class,tile_id,overlay_id,dtm_elevation,dtm_slope,landsat_ard_spring_blue_p25,landsat_ard_winter_green_p75,...,landsat_ard_summer_thermal_p75,landsat_ard_fall_thermal_p75,landsat_ard_spring_thermal_p50,landsat_ard_spring_swir2_p75,landsat_ard_spring_thermal_p25,landsat_ard_winter_thermal_p75,landsat_ard_winter_thermal_p25,landsat_ard_fall_thermal_p50,night_lights,geometry
0,True,2012-05-29T00:00:00,100,312,22497,1,239.0,2.946278,2.0,8.0,...,183.0,183.0,183.0,8.0,183.0,183.0,183.0,183.0,0.059711,POINT (4650000.166 4483999.711)
1,True,2012-05-16T00:00:00,100,312,22497,2,391.0,5.559027,2.0,8.0,...,183.0,183.0,182.0,7.0,182.0,183.0,182.0,183.0,0.009072,POINT (4650000.255 4471999.472)
2,False,2012-06-30T00:00:00,85,411,22497,3,416.0,1.666667,9.0,15.0,...,185.0,184.0,184.0,28.0,183.0,184.0,184.0,184.0,0.030172,POINT (4650097.582 4470351.405)
3,False,2012-06-30T00:00:00,85,312,22497,4,357.0,8.700255,2.0,7.0,...,182.0,183.0,181.0,6.0,181.0,182.0,182.0,183.0,-0.012674,POINT (4651001.339 4472046.880)
4,False,2012-06-30T00:00:00,85,312,22497,5,133.0,23.109041,3.0,8.0,...,184.0,183.0,183.0,9.0,183.0,183.0,183.0,183.0,0.200448,POINT (4651217.720 4488931.650)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,True,2015-10-29T00:00:00,100,312,22497,6,207.0,17.834112,2.0,8.0,...,185.0,183.0,183.0,5.0,183.0,183.0,183.0,183.0,0.859678,POINT (4668000.057 4493999.655)
676,True,2015-10-29T00:00:00,100,312,22497,7,172.0,21.122986,5.0,8.0,...,183.0,183.0,183.0,5.0,183.0,183.0,183.0,183.0,0.925818,POINT (4670000.156 4491999.887)
677,True,2015-07-27T00:00:00,100,511,22497,8,113.0,0.000000,2.0,6.0,...,183.0,184.0,181.0,1.0,181.0,183.0,182.0,183.0,0.900600,POINT (4662000.000 4478000.000)
678,True,2015-10-29T00:00:00,100,311,22497,9,40.0,15.023130,2.0,5.0,...,183.0,183.0,182.0,2.0,182.0,183.0,182.0,183.0,2.580281,POINT (4676000.000 4488000.000)


What are the columns avaiable to do machine learning ?

In [28]:
print("Columns:")
columns = []
for col_name, col_type in zip(points.columns, points.dtypes):
    print(f' - {col_name} ({col_type})')

Columns:
 - lucas (bool)
 - survey_date (object)
 - confidence (object)
 - lc_class (int64)
 - tile_id (int64)
 - overlay_id (int64)
 - dtm_elevation (float64)
 - dtm_slope (float64)
 - landsat_ard_spring_blue_p25 (float64)
 - landsat_ard_winter_green_p75 (float64)
 - landsat_ard_winter_blue_p25 (float64)
 - landsat_ard_spring_blue_p50 (float64)
 - landsat_ard_fall_blue_p25 (float64)
 - landsat_ard_winter_blue_p75 (float64)
 - landsat_ard_fall_green_p50 (float64)
 - landsat_ard_summer_blue_p25 (float64)
 - landsat_ard_fall_blue_p75 (float64)
 - landsat_ard_fall_green_p25 (float64)
 - landsat_ard_spring_green_p25 (float64)
 - landsat_ard_summer_red_p25 (float64)
 - landsat_ard_spring_blue_p75 (float64)
 - landsat_ard_summer_green_p50 (float64)
 - landsat_ard_summer_blue_p50 (float64)
 - landsat_ard_fall_blue_p50 (float64)
 - landsat_ard_winter_green_p25 (float64)
 - landsat_ard_summer_green_p25 (float64)
 - landsat_ard_spring_red_p50 (float64)
 - landsat_ard_spring_red_p25 (float64)
 - 

In [29]:
feat_col_prfxs = ['landsat', 'dtm', 'night_lights']
target_col = 'lc_class'
estimator = RandomForestClassifier(n_estimators=100)

landmapper = LandMapper(fn_points, feat_col_prfxs, target_col, estimator=estimator, 
                        val_samples_pct=0.5, min_samples_per_class=0.05)

[11:39:52] Removing 60 sampes due min_samples_per_class = 0.05
[11:39:52] train_val_split is a invalid validation strategy


# Training 

In [30]:
landmapper.train()

[11:39:54] Training and evaluating the model
[11:39:55] Training the final model using all data


In [31]:
print(f'Overall accuracy: {landmapper.overall_acc * 100:.2f}%')
landmapper.cm

Overall accuracy: 90.97%


array([[61,  0,  3,  1,  0,  0],
       [ 0, 18,  3,  1,  0,  0],
       [ 1,  3, 81,  1,  0,  0],
       [ 0,  0,  4, 25,  2,  0],
       [ 1,  0,  7,  0, 55,  0],
       [ 0,  0,  1,  0,  0, 42]])

# Predicting one year

In [23]:
dir_timeless_layers = os.path.join(data_dir, 'images/timeless') 
dir_2000_layers = os.path.join(data_dir, 'images/2000')
fn_result = os.path.join(data_dir, f'land_cover_2000.tif')

landmapper.predict([dir_2000_layers, dir_timeless_layers], fn_result)

[11:08:09] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_winter_green_p50.tif
[11:08:09] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_summer_blue_p50.tif
[11:08:09] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_fall_green_p75.tif
[11:08:09] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_summer_red_p25.tif
[11:08:09] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_winter_blue_p25.tif
[11:08:09] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_spring_green_p50.tif
[11:08:09] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_summer_blue_p75.tif
[11:08:09] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_summer_green_p25.tif
[11:08:10] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_winter_blue_p75.tif
[11:08:10] Reading /home/leandro/Tmp/pilot_ti

[11:08:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_fall_thermal_p25.tif
[11:08:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_fall_thermal_p75.tif
[11:08:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_spring_thermal_p50.tif
[11:08:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_spring_thermal_p75.tif
[11:08:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_winter_thermal_p75.tif
[11:08:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/landsat_ard_summer_thermal_p75.tif
[11:08:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/timeless/dtm_elevation.tif
[11:08:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/timeless/dtm_slope.tif
[11:08:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2000/night_lights.tif
[11:08:13] Predicing 1000000 pixels
[11:08:27] Saving the result in /ho

# Predicting several years

In [24]:
dir_timeless_layers = os.path.join(data_dir, 'images/timeless')

for year in range(2001, 2020):
    dir_time_layers = os.path.join(data_dir, 'images', str(year))
    fn_result = os.path.join(data_dir, f'land_cover_{year}.tif')
    
    landmapper.predict([dir_time_layers, dir_timeless_layers], fn_result)

[11:09:45] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_winter_green_p50.tif
[11:09:45] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_summer_blue_p50.tif
[11:09:45] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_fall_green_p75.tif
[11:09:45] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_summer_red_p25.tif
[11:09:45] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_winter_blue_p25.tif
[11:09:45] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_spring_green_p50.tif
[11:09:45] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_summer_blue_p75.tif
[11:09:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_summer_green_p25.tif
[11:09:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_winter_blue_p75.tif
[11:09:46] Reading /home/leandro/Tmp/pilot_ti

[11:09:47] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_fall_thermal_p25.tif
[11:09:47] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_fall_thermal_p75.tif
[11:09:47] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_spring_thermal_p50.tif
[11:09:47] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_spring_thermal_p75.tif
[11:09:47] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_winter_thermal_p75.tif
[11:09:47] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/landsat_ard_summer_thermal_p75.tif
[11:09:47] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/timeless/dtm_elevation.tif
[11:09:47] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/timeless/dtm_slope.tif
[11:09:47] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2001/night_lights.tif
[11:09:49] Predicing 1000000 pixels
[11:10:03] Saving the result in /ho

[11:10:04] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2002/landsat_ard_summer_swir1_p75.tif
[11:10:04] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2002/landsat_ard_summer_thermal_p25.tif
[11:10:04] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2002/landsat_ard_spring_swir2_p50.tif
[11:10:04] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2002/landsat_ard_spring_swir2_p75.tif
[11:10:04] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2002/landsat_ard_fall_thermal_p50.tif
[11:10:04] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2002/landsat_ard_summer_swir2_p75.tif
[11:10:04] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2002/landsat_ard_winter_thermal_p50.tif
[11:10:04] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2002/landsat_ard_winter_swir2_p75.tif
[11:10:04] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2002/landsat_ard_summer_thermal_p50.tif
[11:10:04] Reading /home/leandr

[11:10:22] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2003/landsat_ard_summer_nir_p75.tif
[11:10:22] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2003/landsat_ard_spring_swir2_p25.tif
[11:10:22] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2003/landsat_ard_summer_swir2_p25.tif
[11:10:22] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2003/landsat_ard_winter_swir2_p25.tif
[11:10:22] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2003/landsat_ard_summer_swir1_p50.tif
[11:10:22] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2003/landsat_ard_fall_swir2_p50.tif
[11:10:22] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2003/landsat_ard_spring_thermal_p25.tif
[11:10:22] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2003/landsat_ard_winter_swir1_p75.tif
[11:10:22] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2003/landsat_ard_winter_thermal_p25.tif
[11:10:22] Reading /home/leandro/Tmp/

[11:10:39] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2004/landsat_ard_summer_swir1_p25.tif
[11:10:39] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2004/landsat_ard_spring_swir1_p75.tif
[11:10:39] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2004/landsat_ard_fall_swir2_p25.tif
[11:10:39] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2004/landsat_ard_spring_nir_p75.tif
[11:10:39] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2004/landsat_ard_fall_nir_p25.tif
[11:10:39] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2004/landsat_ard_fall_swir1_p50.tif
[11:10:39] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2004/landsat_ard_spring_swir1_p50.tif
[11:10:39] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2004/landsat_ard_spring_swir1_p25.tif
[11:10:39] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2004/landsat_ard_fall_swir1_p75.tif
[11:10:39] Reading /home/leandro/Tmp/pilot_tiles/

...

[11:14:10] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2016/landsat_ard_summer_blue_p75.tif
[11:14:10] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2016/landsat_ard_summer_green_p25.tif
[11:14:10] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2016/landsat_ard_winter_blue_p75.tif
[11:14:10] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2016/landsat_ard_spring_blue_p25.tif
[11:14:10] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2016/landsat_ard_winter_blue_p50.tif
[11:14:10] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2016/landsat_ard_summer_green_p75.tif
[11:14:10] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2016/landsat_ard_winter_green_p25.tif
[11:14:10] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2016/landsat_ard_spring_red_p25.tif
[11:14:10] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2016/landsat_ard_summer_blue_p25.tif
[11:14:10] Reading /home/leandro/Tmp/pilot_t

[11:14:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/timeless/dtm_elevation.tif
[11:14:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/timeless/dtm_slope.tif
[11:14:11] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2016/night_lights.tif
[11:14:13] Predicing 1000000 pixels
[11:14:27] Saving the result in /home/leandro/Tmp/pilot_tiles/croatia_9529/land_cover_2016.tif
[11:14:27] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_winter_green_p50.tif
[11:14:27] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_summer_blue_p50.tif
[11:14:27] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_fall_green_p75.tif
[11:14:27] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_summer_red_p25.tif
[11:14:27] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_winter_blue_p25.tif
[11:14:27] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529

[11:14:29] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_winter_thermal_p50.tif
[11:14:29] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_winter_swir2_p75.tif
[11:14:29] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_summer_thermal_p50.tif
[11:14:29] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_winter_swir2_p50.tif
[11:14:29] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_fall_thermal_p25.tif
[11:14:29] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_fall_thermal_p75.tif
[11:14:29] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_spring_thermal_p50.tif
[11:14:29] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_spring_thermal_p75.tif
[11:14:29] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2017/landsat_ard_winter_thermal_p75.tif
[11:14:29] Reading /home/le

[11:14:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2018/landsat_ard_fall_swir2_p50.tif
[11:14:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2018/landsat_ard_spring_thermal_p25.tif
[11:14:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2018/landsat_ard_winter_swir1_p75.tif
[11:14:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2018/landsat_ard_winter_thermal_p25.tif
[11:14:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2018/landsat_ard_fall_swir2_p75.tif
[11:14:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2018/landsat_ard_summer_swir1_p75.tif
[11:14:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2018/landsat_ard_summer_thermal_p25.tif
[11:14:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2018/landsat_ard_spring_swir2_p50.tif
[11:14:46] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2018/landsat_ard_spring_swir2_p75.tif
[11:14:46] Reading /home/leandro/Tm

[11:15:03] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2019/landsat_ard_fall_swir1_p50.tif
[11:15:03] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2019/landsat_ard_spring_swir1_p50.tif
[11:15:03] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2019/landsat_ard_spring_swir1_p25.tif
[11:15:03] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2019/landsat_ard_fall_swir1_p75.tif
[11:15:03] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2019/landsat_ard_summer_swir2_p50.tif
[11:15:03] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2019/landsat_ard_summer_nir_p75.tif
[11:15:03] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2019/landsat_ard_spring_swir2_p25.tif
[11:15:03] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2019/landsat_ard_summer_swir2_p25.tif
[11:15:03] Reading /home/leandro/Tmp/pilot_tiles/croatia_9529/images/2019/landsat_ard_winter_swir2_p25.tif
[11:15:03] Reading /home/leandro/Tmp/pilot_

## Visualization

In [10]:
# http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/
# https://stackoverflow.com/questions/43445103/inline-animations-in-jupyter
#import rasterio 
#from rasterio.plot import show
#import matplotlib.pyplot as plt
#from matplotlib.colors import ListedColormap, BoundaryNorm

#img = rasterio.open(fn_result)

#colors = ['linen', 'lightgreen', 'lightgreen', 'lightgreen', 'lightgreen', 'lightgreen']
#cmap = ListedColormap(colors)
#norm = BoundaryNorm([211, 311, 312, 324, 411, 512], len(colors))

#show(img, cmap=cmap, norm=norm, title='2000')
#show(img, cmap=cmap, norm=norm, title='2001')